#STEP 1 — Open Dataset Safely

In [3]:
import pandas as pd

# Load only 100k rows (safe for laptop)
df = pd.read_csv("flights.csv", nrows=100000)

# View first rows
print(df.head())


      FL_DATE                AIRLINE                AIRLINE_DOT AIRLINE_CODE  \
0  2019-01-09  United Air Lines Inc.  United Air Lines Inc.: UA           UA   
1  2022-11-19   Delta Air Lines Inc.   Delta Air Lines Inc.: DL           DL   
2  2022-07-22  United Air Lines Inc.  United Air Lines Inc.: UA           UA   
3  2023-03-06   Delta Air Lines Inc.   Delta Air Lines Inc.: DL           DL   
4  2020-02-23       Spirit Air Lines       Spirit Air Lines: NK           NK   

   DOT_CODE  FL_NUMBER ORIGIN          ORIGIN_CITY DEST  \
0   19977.0     1562.0    FLL  Fort Lauderdale, FL  EWR   
1   19790.0     1149.0    MSP      Minneapolis, MN  SEA   
2   19977.0      459.0    DEN           Denver, CO  MSP   
3   19790.0     2295.0    MSP      Minneapolis, MN  SFO   
4   20416.0      407.0    MCO          Orlando, FL  DFW   

               DEST_CITY  ...  DIVERTED  CRS_ELAPSED_TIME  ELAPSED_TIME  \
0             Newark, NJ  ...       0.0             186.0         176.0   
1            S

#STEP 2 — Understand Dataset

In [4]:
# Structure of data
print(df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15361 entries, 0 to 15360
Data columns (total 32 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   FL_DATE                  15361 non-null  object 
 1   AIRLINE                  15361 non-null  object 
 2   AIRLINE_DOT              15361 non-null  object 
 3   AIRLINE_CODE             15361 non-null  object 
 4   DOT_CODE                 15360 non-null  float64
 5   FL_NUMBER                15360 non-null  float64
 6   ORIGIN                   15360 non-null  object 
 7   ORIGIN_CITY              15360 non-null  object 
 8   DEST                     15360 non-null  object 
 9   DEST_CITY                15360 non-null  object 
 10  CRS_DEP_TIME             15360 non-null  float64
 11  DEP_TIME                 14990 non-null  float64
 12  DEP_DELAY                14990 non-null  float64
 13  TAXI_OUT                 14986 non-null  float64
 14  WHEELS_OFF            

In [5]:
# Basic stats
print(df.describe())

           DOT_CODE     FL_NUMBER  CRS_DEP_TIME      DEP_TIME     DEP_DELAY  \
count  15360.000000  15360.000000  15360.000000  14990.000000  14990.000000   
mean   19973.896484   2517.907357   1324.423893   1329.045430      9.705937   
std      378.268293   1745.077372    488.451016    502.116648     46.464559   
min    19393.000000      1.000000      7.000000      1.000000    -41.000000   
25%    19790.000000   1064.000000    910.000000    911.000000     -5.000000   
50%    19930.000000   2161.000000   1320.000000   1323.000000     -2.000000   
75%    20368.000000   3793.000000   1730.000000   1742.000000      7.000000   
max    20452.000000   8799.000000   2359.000000   2400.000000   1180.000000   

           TAXI_OUT    WHEELS_OFF     WHEELS_ON       TAXI_IN  CRS_ARR_TIME  \
count  14986.000000  14986.000000  14982.000000  14982.000000  15360.000000   
mean      16.575938   1350.728547   1463.873381      7.726005   1488.665039   
std        9.229821    503.677466    529.975260    

In [6]:
# Check missing values
print(df.isnull().sum())

FL_DATE                        0
AIRLINE                        0
AIRLINE_DOT                    0
AIRLINE_CODE                   0
DOT_CODE                       1
FL_NUMBER                      1
ORIGIN                         1
ORIGIN_CITY                    1
DEST                           1
DEST_CITY                      1
CRS_DEP_TIME                   1
DEP_TIME                     371
DEP_DELAY                    371
TAXI_OUT                     375
WHEELS_OFF                   375
WHEELS_ON                    379
TAXI_IN                      379
CRS_ARR_TIME                   1
ARR_TIME                     379
ARR_DELAY                    405
CANCELLED                      1
CANCELLATION_CODE          14986
DIVERTED                       1
CRS_ELAPSED_TIME               1
ELAPSED_TIME                 405
AIR_TIME                     405
DISTANCE                       1
DELAY_DUE_CARRIER          12634
DELAY_DUE_WEATHER          12634
DELAY_DUE_NAS              12634
DELAY_DUE_

📌 Data Quality Observations
#1️⃣ Date Column

FL_DATE is stored as an object (string) datatype.
This column needs to be converted into datetime format for proper time-based analysis and feature extraction.

➡ Action Taken: Converted to datetime format.

#2️⃣ Delay Columns with Missing Values

DEP_DELAY contains 371 missing values.
ARR_DELAY contains 405 missing values.

Missing values in delay columns can affect delay analysis and statistics.

➡ Action Taken: Filled missing values using the median of each column.

#3️⃣ Cancellation Code Mostly Empty

CANCELLATION_CODE has 14,986 missing values.
This indicates that most flights were not cancelled.

➡ Action Taken: Filled missing values with "Not Cancelled".

#4️⃣ Delay Reason Columns Mostly Empty

Delay reason columns such as:

DELAY_DUE_CARRIER

DELAY_DUE_WEATHER

DELAY_DUE_NAS

DELAY_DUE_SECURITY

DELAY_DUE_LATE_AIRCRAFT

have many missing values.

These missing values usually indicate that no delay occurred for that reason.

➡ Action Taken: Filled missing values with 0.

#STEP 3 — Fix Date Format

In [7]:
df['FL_DATE'] = pd.to_datetime(df['FL_DATE'])


In [8]:
print(df['FL_DATE'].head())


0   2019-01-09
1   2022-11-19
2   2022-07-22
3   2023-03-06
4   2020-02-23
Name: FL_DATE, dtype: datetime64[ns]


#STEP 4 — Handle Missing Values
**Fill delay columns with median**

In [9]:
df['DEP_DELAY'] = df['DEP_DELAY'].fillna(df['DEP_DELAY'].median())
df['ARR_DELAY'] = df['ARR_DELAY'].fillna(df['ARR_DELAY'].median())


**Fill cancellation code**

In [10]:
df['CANCELLATION_CODE'] = df['CANCELLATION_CODE'].fillna("Not Cancelled")


**Fill delay reasons with 0**

In [11]:
delay_cols = [
    'DELAY_DUE_CARRIER',
    'DELAY_DUE_WEATHER',
    'DELAY_DUE_NAS',
    'DELAY_DUE_SECURITY',
    'DELAY_DUE_LATE_AIRCRAFT'
]

df[delay_cols] = df[delay_cols].fillna(0)


#STEP 5 — Feature Engineering

**Month**

In [12]:
df['MONTH'] = df['FL_DATE'].dt.month


**Day of Week**

In [13]:
df['DAY_OF_WEEK'] = df['FL_DATE'].dt.dayofweek


**Hour (from DEP_TIME)**-The
**time is in HHMM format.**

In [14]:
df['DEP_TIME'] = pd.to_datetime(df['DEP_TIME'], format='%H%M', errors='coerce')

df['HOUR'] = df['DEP_TIME'].dt.hour


**Route Feature**

In [15]:
df['ROUTE'] = df['ORIGIN'] + "-" + df['DEST']


#STEP 6 — Remove Duplicates

In [16]:
df.drop_duplicates(inplace=True)


#STEP 7 — Ensure Positive Delays

In [17]:
df = df[df['ARR_DELAY'] >= 0]
df = df[df['DEP_DELAY'] >= 0]


#STEP 8 — Normalize Airline Names

In [18]:
df['AIRLINE'] = df['AIRLINE'].str.upper().str.strip()


#Step 9- Data Quality & Statistical Validation

In [19]:
print("Final Dataset Shape:", df.shape)

Final Dataset Shape: (3944, 36)


In [20]:
print(df.isnull().sum())


FL_DATE                    0
AIRLINE                    0
AIRLINE_DOT                0
AIRLINE_CODE               0
DOT_CODE                   0
FL_NUMBER                  0
ORIGIN                     0
ORIGIN_CITY                0
DEST                       0
DEST_CITY                  0
CRS_DEP_TIME               0
DEP_TIME                   8
DEP_DELAY                  0
TAXI_OUT                   0
WHEELS_OFF                 0
WHEELS_ON                  0
TAXI_IN                    0
CRS_ARR_TIME               0
ARR_TIME                   0
ARR_DELAY                  0
CANCELLED                  0
CANCELLATION_CODE          0
DIVERTED                   0
CRS_ELAPSED_TIME           0
ELAPSED_TIME               0
AIR_TIME                   0
DISTANCE                   0
DELAY_DUE_CARRIER          0
DELAY_DUE_WEATHER          0
DELAY_DUE_NAS              0
DELAY_DUE_SECURITY         0
DELAY_DUE_LATE_AIRCRAFT    0
MONTH                      0
DAY_OF_WEEK                0
HOUR          

**Skewness**

Skewness measures:The asymmetry of a data distribution.
It tells whether data is tilted more to the left or right.

In [21]:
print("Arrival Delay Skewness:", df['ARR_DELAY'].skew())
print("Departure Delay Skewness:", df['DEP_DELAY'].skew())

Arrival Delay Skewness: 6.553238482445247
Departure Delay Skewness: 6.4942599265696


distribution is heavily right-skewed.
This means most flights Have small delay Or near zero delay But Few flights have very huge delays

Like:

200 minutes

400 minutes

800 minutes

These extreme values pull the distribution to the right.

Positive skew confirms presence of extreme delay values.

**Mean vs Median**

In [22]:
print("Mean:", df['ARR_DELAY'].mean())
print("Median:", df['ARR_DELAY'].median())

Mean: 45.2789046653144
Median: 22.0


mean > median:Confirms right-skewed distribution.

**99th Percentile**

Finds value below which 99% of data lies. This is Used to understand extreme boundary values before defining outliers.

In [23]:
print("99th Percentile ARR_DELAY:",
df['ARR_DELAY'].quantile(0.99))

99th Percentile ARR_DELAY: 326.0


**Early Arrival Count**

Negative delay indicates early arrival, which is operationally meaningful.

In [24]:
print("Flights Arrived Early:",
(df['ARR_DELAY'] < 0).sum())

Flights Arrived Early: 0


**Correlation**

In [26]:
df[['DEP_DELAY','ARR_DELAY']].corr()

,DEP_DELAY,ARR_DELAY
DEP_DELAY,1.000000,0.981016
ARR_DELAY,0.981016,1.000000


The correlation coefficient of 0.98 indicates a very strong positive linear relationship between departure delay and arrival delay, suggesting that departure delay is a strong predictor of arrival delay.

**Basic Aggregation**

In [27]:
df.groupby('AIRLINE')['ARR_DELAY'].mean()

,ARR_DELAY
AIRLINE,
ALASKA AIRLINES INC.,29.927273
ALLEGIANT AIR,55.694118
AMERICAN AIRLINES INC.,50.611219
DELTA AIR LINES INC.,46.742004
ENDEAVOR AIR INC.,52.812500
ENVOY AIR,35.246032
EXPRESSJET AIRLINES LLC D/B/A AHA!,30.476190
FRONTIER AIRLINES INC.,57.333333
HAWAIIAN AIRLINES INC.,27.224138


**Delay Percentage**

In [28]:
print("Flights Delayed >15 mins (%):",
      (df['ARR_DELAY'] > 15).mean() * 100)

Flights Delayed >15 mins (%): 60.06592292089249


**Cancellation Rate**

In [29]:
print("Cancellation Rate (%):",
      df['CANCELLED'].mean() * 100)

Cancellation Rate (%): 0.0


**Categorical Diversity Check**

In [31]:
print("Unique Airlines:", df['AIRLINE'].nunique())
print("Unique Origin Airports:", df['ORIGIN'].nunique())
print("Unique Routes:", df['ROUTE'].nunique())

Unique Airlines: 18
Unique Origin Airports: 243
Unique Routes: 2223


**Route Level Aggregation**

In [34]:
route_agg = df.groupby('ROUTE').agg({
    'DEP_DELAY': 'mean',
    'ARR_DELAY': 'mean',
    'FL_DATE': 'count'
}).rename(columns={'FL_DATE': 'TOTAL_FLIGHTS'})
route_agg = route_agg.round(2)
route_agg.head()

,DEP_DELAY,ARR_DELAY,TOTAL_FLIGHTS
ROUTE,,,
ABE-BNA,4.00,6.00,1
ABQ-DAL,17.75,23.25,4
ABQ-DEN,72.00,61.00,2
ABQ-DFW,115.33,106.67,3
ABQ-HOU,0.00,2.00,1


In [35]:
#These are the top 10 busiest routes
route_agg.sort_values(by='TOTAL_FLIGHTS', ascending=False).head(10)

,DEP_DELAY,ARR_DELAY,TOTAL_FLIGHTS
ROUTE,,,
DFW-LAX,163.30,163.70,10
ATL-MCO,24.00,20.10,10
LGA-ORD,114.10,113.60,10
DFW-PHX,72.11,75.78,9
HNL-ITO,13.89,16.00,9
MCO-EWR,31.67,38.56,9
ATL-LGA,31.67,26.78,9
HNL-OGG,18.89,19.11,9
ORD-LGA,64.89,65.33,9


In [36]:
#These routes have the highest average arrival delay.
route_agg.sort_values(by='ARR_DELAY', ascending=False).head(10)

,DEP_DELAY,ARR_DELAY,TOTAL_FLIGHTS
ROUTE,,,
RSW-CLT,1035.0,1020.0,1
JAC-SLC,874.0,866.0,1
CLT-PNS,769.0,796.0,1
XNA-ORD,808.0,793.0,1
ORD-SPI,630.0,609.0,1
BUF-JFK,538.0,574.0,1
ESC-DTW,484.0,480.0,1
AUS-PHX,479.0,466.0,1
AUS-DEN,468.0,457.0,2


In [37]:
route_agg.shape

(2223, 3)

**Basic Airline Aggregation**

In [38]:
#Provides average arriving delay per airline for validation.
df.groupby('AIRLINE')['ARR_DELAY'].mean().head()

,ARR_DELAY
AIRLINE,
ALASKA AIRLINES INC.,29.927273
ALLEGIANT AIR,55.694118
AMERICAN AIRLINES INC.,50.611219
DELTA AIR LINES INC.,46.742004
ENDEAVOR AIR INC.,52.812500


**Monthly Aggregation**

In [40]:
#Checks if arriving delay varies across months.
df.groupby('MONTH')['ARR_DELAY'].mean()

,ARR_DELAY
MONTH,
1,52.742947
2,49.812925
3,42.540785
4,43.369906
5,45.021807
6,50.454352
7,46.237443
8,43.763224
9,48.926108


**Delay Reason Aggregation**

In [42]:
#Each column represents minutes delayed due to that reason.
#.sum() works column-wise.So it adds all rows in each column.
#This sorts reasons from highest total delay to lowest.
delay_reason_totals = df[['DELAY_DUE_CARRIER',
                          'DELAY_DUE_WEATHER',
                          'DELAY_DUE_NAS',
                          'DELAY_DUE_SECURITY',
                          'DELAY_DUE_LATE_AIRCRAFT']].sum()

delay_reason_totals.sort_values(ascending=False)

,0
DELAY_DUE_CARRIER,65395.0
DELAY_DUE_LATE_AIRCRAFT,65215.0
DELAY_DUE_NAS,26993.0
DELAY_DUE_WEATHER,11251.0
DELAY_DUE_SECURITY,470.0


#STEP 10 — Save Cleaned File

In [30]:
print(df.isnull().sum())


FL_DATE                    0
AIRLINE                    0
AIRLINE_DOT                0
AIRLINE_CODE               0
DOT_CODE                   0
FL_NUMBER                  0
ORIGIN                     0
ORIGIN_CITY                0
DEST                       0
DEST_CITY                  0
CRS_DEP_TIME               0
DEP_TIME                   8
DEP_DELAY                  0
TAXI_OUT                   0
WHEELS_OFF                 0
WHEELS_ON                  0
TAXI_IN                    0
CRS_ARR_TIME               0
ARR_TIME                   0
ARR_DELAY                  0
CANCELLED                  0
CANCELLATION_CODE          0
DIVERTED                   0
CRS_ELAPSED_TIME           0
ELAPSED_TIME               0
AIR_TIME                   0
DISTANCE                   0
DELAY_DUE_CARRIER          0
DELAY_DUE_WEATHER          0
DELAY_DUE_NAS              0
DELAY_DUE_SECURITY         0
DELAY_DUE_LATE_AIRCRAFT    0
MONTH                      0
DAY_OF_WEEK                0
HOUR          

In [43]:
df.to_csv("cleaned_flights.csv", index=False)
